# **데이터 전처리**

In [10]:
!pip install TA-lib

In [11]:
!pip install backtrader

In [12]:
!pip install pygad

In [1]:
import talib
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


from FNC_02_Preprocessing import FNC_02_Preprocessing
from FNC_Func_Extrema import FNC_Func_Extrema
from FNC_Func_PST import FNC_Func_PST

import matplotlib.pyplot as plt

In [69]:
%cd "./data_u"

[WinError 2] The system cannot find the file specified: './data_u'
c:\Users\ccn-team\Desktop\BXAI\stockdata\data_u


In [70]:
import os
a = os.listdir()
a

['KR7000270009.csv',
 'KR7000660001.csv',
 'KR7003670007.csv',
 'KR7005380001.csv',
 'KR7005490008.csv',
 'KR7005930003.csv',
 'KR7006400006.csv',
 'KR7035420009.csv',
 'KR7051910008.csv',
 'KR7207940008.csv',
 'results.csv']

In [71]:
def bt_data(data_bt):
    check_dtype = data_bt.dtype == 'object'
    if (check_dtype):
        return data_bt.str.replace(',','').astype('float')
    else :
        return data_bt.astype('float')

In [72]:
import pygad
import backtrader
import pandas as pd
import talib

class Trading:
    def __init__(self, df):
        self.df = df.copy()
        self.initial_capital = 100000.0

    def rsi(self,ga_instance, solution, solution_idx ):
        period, rsi_low, rsi_high = solution
        self.df['rsi'] = talib.RSI(self.df['Marketcap'], timeperiod=int(period))
        self.df['signal'] = np.where(self.df['rsi'] < rsi_low, 1, np.where(self.df['rsi'] > rsi_high, -1, 0))
        return self.trade_logic()

    def sma_cross(self,ga_instance, solution, solution_idx):
        pfast, pslow = solution
        self.df['sma1'] = talib.SMA(self.df['Marketcap'], timeperiod=int(pfast))
        self.df['sma2'] = talib.SMA(self.df['Marketcap'], timeperiod=int(pslow))
        self.df['signal'] = np.where(self.df['sma1'] > self.df['sma2'], 1, -1)
        return self.trade_logic()

    def ema_cross(self, ga_instance, solution, solution_idx):
        pfast, pslow=solution
        self.df['ema1'] = talib.EMA(self.df['Marketcap'], timeperiod=int(pfast))
        self.df['ema2'] = talib.EMA(self.df['Marketcap'], timeperiod=int(pslow))
        self.df['signal'] = np.where(self.df['ema1'] > self.df['ema2'], 1, -1)
        return self.trade_logic()

    def roc(self,ga_instance, solution, solution_idx):
        period = solution
        self.df['roc'] = talib.ROC(self.df['Marketcap'], timeperiod=int(period))
        self.df['signal'] = np.where(self.df['roc'] > 0, 1, -1)
        return self.trade_logic()

    def stochastic(self,ga_instance, solution, solution_idx):
        period, d_period, overbought, oversold = solution

        k, d = talib.STOCH(self.df['High']*self.df['Marketcap']/self.df['Close'], self.df['Low']*self.df['Marketcap']/self.df['Close'], self.df['Marketcap'], fastk_period=int(period), slowk_period=int(d_period))
        self.df['signal'] = np.where(k < oversold, 1, np.where(k > overbought, -1, 0))
        return self.trade_logic()

    def trade_logic(self):
        shares = 0
        for i in range(1, len(self.df)):
            if self.df['signal'].iloc[i] == 1 and self.initial_capital > 0:
                if self.df['Marketcap'].iloc[i] <= self.initial_capital:
                    shares = self.initial_capital / self.df['Marketcap'].iloc[i]
                    self.initial_capital =0.0
            elif self.df['signal'].iloc[i] == -1 and shares > 0:
                self.initial_capital += shares * self.df['Marketcap'].iloc[i]
                shares = 0.0

        final_portfolio_value = self.initial_capital + shares * self.df['Marketcap'].iloc[-1]
        return_rate = (final_portfolio_value - self.initial_capital) / self.initial_capital

        return final_portfolio_value

In [73]:
import talib
import numpy as np

class SHOW_SIGNAL:
    def __init__(self, df):
        self.df = df.copy()

    def rsi(self, period, rsi_low, rsi_high):
        self.df['rsi'] = talib.RSI(self.df['Marketcap'], timeperiod=int(period))
        self.df['signal'] = np.where(self.df['rsi'] < rsi_low, 1, np.where(self.df['rsi'] > rsi_high, -1, 0))
        return self.trade_logic()

    def sma_cross(self, pfast, pslow):
        self.df['sma1'] = talib.SMA(self.df['Marketcap'], timeperiod=int(pfast))
        self.df['sma2'] = talib.SMA(self.df['Marketcap'], timeperiod=int(pslow))
        self.df['signal'] = np.where(self.df['sma1'] > self.df['sma2'], 1, -1)
        return self.trade_logic()

    def ema_cross(self, pfast, pslow):
        self.df['ema1'] = talib.EMA(self.df['Marketcap'], timeperiod=int(pfast))
        self.df['ema2'] = talib.EMA(self.df['Marketcap'], timeperiod=int(pslow))
        self.df['signal'] = np.where(self.df['ema1'] > self.df['ema2'], 1, -1)
        return self.trade_logic()

    def roc(self, period):
        self.df['roc'] = talib.ROC(self.df['Marketcap'], timeperiod=int(period))
        self.df['signal'] = np.where(self.df['roc'] > 0, 1, -1)
        return self.trade_logic()

    def stochastic(self, period, d_period, overbought, oversold):
        k, d = talib.STOCH(self.df['High']*self.df['Marketcap']/self.df['Close'], self.df['Low']*self.df['Marketcap']/self.df['Close'], self.df['Marketcap'], fastk_period=int(period), slowk_period=int(d_period))
        self.df['signal'] = np.where(k < oversold, 1, np.where(k > overbought, -1, 0))
        return self.trade_logic()

    def trade_logic(self, initial_capital=100000.0):
        shares = 0
        for i in range(1, len(self.df)):
            if self.df['signal'].iloc[i] == 1 and initial_capital > 0:
                if self.df['Marketcap'].iloc[i] <= initial_capital:
                    shares = initial_capital / self.df['Marketcap'].iloc[i]
                    initial_capital =0.0
            elif self.df['signal'].iloc[i] == -1 and shares > 0:
                initial_capital += shares * self.df['Marketcap'].iloc[i]
                shares = 0.0

        final_portfolio_value = initial_capital + shares * self.df['Marketcap'].iloc[-1]
        return_rate = (final_portfolio_value - initial_capital) / initial_capital

        return final_portfolio_value

In [74]:
def callback_generation(ga_instance):
    # 최대 적합도 값 가져오기
    max_fitness = ga_instance.best_solution()[1]
    if max_fitness/100000.0 > df_bt['Marketcap'][-1]/df_bt['Marketcap'][0]:
        ga_instance.stop_training = True

In [75]:
def genetic(g_space,Trading,n_gen):
    ga_instance_rsi = pygad.GA(
        num_generations=50,
        num_parents_mating=50,
        fitness_func=Trading, 
        sol_per_pop=100,
        num_genes=n_gen,
        gene_space=g_space,
        parent_selection_type="rank",
        keep_parents=3,
        crossover_type="uniform",
         on_generation=callback_generation,
        mutation_type="random",
        mutation_percent_genes=5,
        mutation_num_genes=1
    )

    ga_instance_rsi.run()

    solution, solution_fitness, solution_idx = ga_instance_rsi.best_solution()
    return solution, solution_fitness

In [76]:
def get_signals(df):
    trading = Trading(df)

    strategies = {
        "rsi": {"period": 14, "rsi_low": 30, "rsi_high": 70},
        "sma_cross": {"pfast": 20, "pslow": 60},
        "ema_cross": {"pfast": 20, "pslow": 60},
        "roc": {"period": 10},
        "stochastic": {"period": 14, "d_period": 3, "overbought": 70, "oversold": 30}
    }
    for i in ["rsi", "sma_cross", "roc", "ema_cross", "stochastic"]:

        if i == "rsi":
            g_space = [range(2, 100), range(1, 100), range(1, 100)]
            rsi_sol, rsi_fitness = genetic(g_space, trading.rsi, 3)
            strategies['rsi'] = {"period": rsi_sol[0], "rsi_low": rsi_sol[1], "rsi_high": rsi_sol[2]}

        elif i ==  "sma_cross":
            g_space = [range(2, 100), range(2, 100)]
            sma_sol, sma_fitness = genetic(g_space, trading.sma_cross, 2)
            strategies['sma_cross'] = {"pfast": sma_sol[0], "pslow": sma_sol[1]}

        elif i == "roc":
            g_space = [range(2, 100)]
            roc_sol, roc_fitness = genetic(g_space, trading.roc, 1)
            strategies['roc'] = {"period": roc_sol[0]}

        elif i == "ema_cross":
            g_space = [range(2, 100), range(2, 100)]
            ema_sol, ema_fitness = genetic(g_space, trading.ema_cross,2)
            strategies['ema_cross'] = {"pfast": ema_sol[0], "pslow": ema_sol[1]}

        elif i == "stochastic":
            g_space = [range(2, 100),range(2, 100), range(2, 100), range(2, 100)]
            sto_sol, sto_fitness = genetic(g_space, trading.stochastic,4)
            strategies["stochastic"] = {"period": sto_sol[0], "d_period": sto_sol[1], "overbought": sto_sol[2], "oversold": sto_sol[3]}

    signal_dataframes = pd.DataFrame()
    show_sig = SHOW_SIGNAL(df)
    for name, params in strategies.items():
        show_sig.df = df.copy()
        getattr(show_sig, name)(**params)
        signal_dataframes[name] = show_sig.df[['signal']].copy()

    return signal_dataframes

In [77]:
cnt = 0
total =0
results = {}
print(len(a))
for i in range(0,len(a)-1):   
    dfdf = pd.read_csv(a[i])
    columns_to_check = ['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC',
                    'TDD_LWPRC', 'MKTCAP', 'ACC_TRDVOL']

    
    if not all(col in dfdf.columns for col in columns_to_check):
        continue
    dfdf = dfdf[['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC',
           'TDD_LWPRC', 'MKTCAP', 'ACC_TRDVOL']]
    for column in dfdf.columns:
        if dfdf[column].dtype == 'object' and column not in ['TRD_DD', 'ISU_NM']:
            try:
                dfdf[column] = dfdf[column].str.replace(',', '').astype('float')
            except ValueError as e:
                print(f" {column}: {e}")
        elif column not in ['TRD_DD', 'ISU_NM']:
            dfdf[column] = dfdf[column].astype('float')
    df =pd.concat([dfdf['TRD_DD'],dfdf['ISU_CD'],dfdf['ISU_NM'],dfdf['TDD_CLSPRC'],dfdf['TDD_OPNPRC'],dfdf['TDD_HGPRC'],dfdf['TDD_LWPRC'],dfdf['MKTCAP'],dfdf['ACC_TRDVOL']],axis=1)
    df = dfdf.sort_values(by=['TRD_DD'])
    df.reset_index(drop=True,inplace=True)

    #인풋 데이터 모양 맞춰주기(backtest에 들어갈 데이터 모양)
    df_bt = df[['TRD_DD','TDD_OPNPRC','TDD_HGPRC','TDD_LWPRC','TDD_CLSPRC', 'ACC_TRDVOL', 'MKTCAP']].copy()
    df_bt['TRD_DD'] = pd.to_datetime(df_bt['TRD_DD'])
    df_bt.rename(columns={'TRD_DD':'Date', 'TDD_OPNPRC':'Open', 'TDD_HGPRC':'High','TDD_LWPRC':'Low','TDD_CLSPRC':'Close', 'ACC_TRDVOL':'Volume', 'MKTCAP':'Marketcap'}, inplace=True)
    df_bt.set_index('Date',drop=True,inplace=True)
    df_bt['Open'] = bt_data(df_bt['Open'])
    df_bt['High'] = bt_data(df_bt['High'])
    df_bt['Low'] = bt_data(df_bt['Low'])
    df_bt['Close'] = bt_data(df_bt['Close'])
    df_bt['Volume'] = bt_data(df_bt['Volume'])
    df_bt['Marketcap'] = bt_data(df_bt['Marketcap'])

    signal_dataframes = get_signals(df_bt)

    data = pd.DataFrame()
    scaler = MinMaxScaler(feature_range=(0, 1))
    result = scaler.fit_transform(np.array(df_bt['Marketcap']).reshape(-1,1))
    data['A2'] = pd.DataFrame(result)

    # PST 파라매터 설정
    T=5  # time interval
    P=0.05  # percentage
    if data['A2'].size == 0 or len(data['A2']) <= 10:
        data['PST'] = []
        data['nPST'] = 0
    else:
        data['PST'], data['nPST'] = FNC_Func_PST(T, P, data['A2'])

    signal_dataframes = signal_dataframes.reset_index()
    signal_dataframes['Trend'] =data['PST']

    signal_dataframe = signal_dataframes.copy()
    df_bb = df_bt.reset_index(drop=True)
    rdf = df_bb['Close']

    signal_dataframes = pd.concat([signal_dataframe,rdf], axis=1)
    signal_dataframes = pd.concat([signal_dataframes, df_bb['Marketcap']],axis=1)
    signal_dataframes = signal_dataframes.rename(columns={'Close':'TDD_CLSPRC'})
    def backtest(df):
        initial_capital = 100000.0  # 초기 자본
        shares = 0  # 주식 보유량

        # 매 거래일의 포트폴리오 가치를 추적할 리스트
        portfolio_values = []

        for _, row in df.iterrows():
            if row['rsi'] == 1 and initial_capital > 0:
                shares = initial_capital / row['Marketcap']  # 현재 자본으로 최대한 매수
                initial_capital = 0.0 # 현금은 0이 됩니다.

            elif row['rsi'] == -1 and shares > 0:
                initial_capital = shares * row['Marketcap']  # 모든 주식을 판매하고 현금을 얻습니다.
                shares = 0.0  # 주식은 0이 됩니다.

            # 매 거래일의 포트폴리오 가치 계산 (현금 + 주식 가치)
            portfolio_value = initial_capital + shares * row['Marketcap']
            portfolio_values.append(portfolio_value)

        df['Portfolio_Value'] = portfolio_values
        return df

    sd = backtest(signal_dataframes)
    rsi_per = sd['Portfolio_Value'].iloc[-1]/sd['Portfolio_Value'][0]*100

    def backtest(df):
        initial_capital = 100000.0  # 초기 자본
        shares = 0  # 주식 보유량

        # 매 거래일의 포트폴리오 가치를 추적할 리스트
        portfolio_values = []

        for _, row in df.iterrows():
            if row['sma_cross'] ==1 and initial_capital > 0:
                shares = initial_capital / row['Marketcap']  # 현재 자본으로 최대한 매수
                initial_capital = 0.0  # 현금은 0이 됩니다.

            elif row['sma_cross'] == -1 and shares > 0:
                initial_capital = shares * row['Marketcap']  # 모든 주식을 판매하고 현금을 얻습니다.
                shares = 0.0  # 주식은 0이 됩니다.

            # 매 거래일의 포트폴리오 가치 계산 (현금 + 주식 가치)
            portfolio_value = initial_capital + shares * row['Marketcap']
            portfolio_values.append(portfolio_value)

        df['Portfolio_Value'] = portfolio_values
        return df

    sd = backtest(signal_dataframes)

    sma_per = sd['Portfolio_Value'].iloc[-1]/sd['Portfolio_Value'][0]*100

    def backtest(df):
        initial_capital = 100000.0  # 초기 자본
        shares = 0  # 주식 보유량

        # 매 거래일의 포트폴리오 가치를 추적할 리스트
        portfolio_values = []

        for _, row in df.iterrows():
            if row['ema_cross'] == 1 and initial_capital > 0:
                shares = initial_capital / row['Marketcap']  # 현재 자본으로 최대한 매수
                initial_capital = 0
            elif row['ema_cross'] == -1 and shares > 0:
                initial_capital = shares * row['Marketcap']  # 모든 주식을 판매하고 현금을 얻습니다.
                shares = 0.0  # 주식은 0이 됩니다.

            # 매 거래일의 포트폴리오 가치 계산 (현금 + 주식 가치)
            portfolio_value = initial_capital + shares * row['Marketcap']
            portfolio_values.append(portfolio_value)

        df['Portfolio_Value'] = portfolio_values
        return df

    sd = backtest(signal_dataframes)

    ema_per = sd['Portfolio_Value'].iloc[-1]/sd['Portfolio_Value'][0]*100

    def backtest(df):
        initial_capital = 100000.0  # 초기 자본
        shares = 0  # 주식 보유량

        # 매 거래일의 포트폴리오 가치를 추적할 리스트
        portfolio_values = []

        for _, row in df.iterrows():
            if row['roc'] == 1 and initial_capital > 0:
                shares = initial_capital / row['Marketcap']  # 현재 자본으로 최대한 매수
                initial_capital = 0

            elif row['roc'] == -1 and shares > 0:
                initial_capital = shares * row['Marketcap']  # 모든 주식을 판매하고 현금을 얻습니다.
                shares = 0.0  # 주식은 0이 됩니다.

            # 매 거래일의 포트폴리오 가치 계산 (현금 + 주식 가치)
            portfolio_value = initial_capital + shares * row['Marketcap']
            portfolio_values.append(portfolio_value)

        df['Portfolio_Value'] = portfolio_values
        return df

    sd = backtest(signal_dataframes)

    roc_per = sd['Portfolio_Value'].iloc[-1]/sd['Portfolio_Value'][0]*100

    def backtest(df):
        initial_capital = 100000.0  # 초기 자본
        shares = 0  # 주식 보유량

        # 매 거래일의 포트폴리오 가치를 추적할 리스트
        portfolio_values = []

        for _, row in df.iterrows():
            if row['stochastic'] == 1 and initial_capital > 0:
                shares = initial_capital / row['Marketcap']  # 현재 자본으로 최대한 매수
                initial_capital = 0

            elif row['stochastic'] == -1 and shares > 0:
                initial_capital = shares * row['Marketcap']  # 모든 주식을 판매하고 현금을 얻습니다.
                shares = 0.0  # 주식은 0이 됩니다.

            # 매 거래일의 포트폴리오 가치 계산 (현금 + 주식 가치)
            portfolio_value = initial_capital + shares * row['Marketcap']
            portfolio_values.append(portfolio_value)

        df['Portfolio_Value'] = portfolio_values
        return df

    sd = backtest(signal_dataframes)

    sto_per = sd['Portfolio_Value'].iloc[-1]/sd['Portfolio_Value'][0]*100

    origin_per = df_bt['Marketcap'][-1]/df_bt['Marketcap'][0]*100
    print(dfdf['ISU_NM'][0],"--   buy_and_hold : {:0.2f}%, rsi : {:0.2f}%, ema : {:0.2f}%, roc : {:0.2f}%, sto : {:0.2f}%".format(origin_per, rsi_per, ema_per, roc_per, sto_per),end ='\t')
    if origin_per<rsi_per or origin_per<ema_per or origin_per<roc_per or origin_per<sto_per:
        print("수익률 : {:0.2f}%".format(max(rsi_per, ema_per, roc_per, sto_per)/origin_per *100),end="") 
        cnt += 1
    isu_name = dfdf['ISU_NM'][0]
    
    # 결과 저장
    results[isu_name] = {
        "buy_and_hold": origin_per,
        "rsi": rsi_per,
        "ema": ema_per,
        "roc": roc_per,
        "sto": sto_per,
        "수익률":max(rsi_per, ema_per, roc_per, sto_per)/origin_per *100
    }
    total+=1
    print("\n")
# 딕셔너리를 DataFrame으로 변환
df_results = pd.DataFrame.from_dict(results, orient='index')

# DataFrame을 CSV 파일로 저장
df_results.to_csv('results.csv')

11
기아 --   buy_and_hold : 3812.57%, rsi : 100.00%, ema : 171.88%, roc : 1219.89%, sto : 30.79%	

SK하이닉스 --   buy_and_hold : 8807.59%, rsi : 4170.44%, ema : 207.94%, roc : 399.67%, sto : 12908.15%	수익률 : 146.56%

포스코케미칼 --   buy_and_hold : 13378.19%, rsi : 100.00%, ema : 343.85%, roc : 5639.56%, sto : 100.00%	

현대차 --   buy_and_hold : 2425.89%, rsi : 100.00%, ema : 317.27%, roc : 1116.03%, sto : 3248.58%	수익률 : 133.91%

POSCO --   buy_and_hold : 425.26%, rsi : 100.00%, ema : 327.84%, roc : 556.96%, sto : 188.35%	수익률 : 130.97%

삼성전자 --   buy_and_hold : 7249.68%, rsi : 11030.82%, ema : 1746.11%, roc : 1592.27%, sto : 2223.46%	수익률 : 152.16%

삼성SDI --   buy_and_hold : 4430.35%, rsi : 119.84%, ema : 504.98%, roc : 388.18%, sto : 3141.90%	

NAVER --   buy_and_hold : 16794.43%, rsi : 9624.15%, ema : 33369.70%, roc : 540.09%, sto : 11811.14%	수익률 : 198.70%

LG화학 --   buy_and_hold : 6186.65%, rsi : 100.00%, ema : 745.95%, roc : 1465.48%, sto : 100.00%	

삼성바이오로직스 --   buy_and_hold : 578.47%, rsi : 5

In [ ]:
cnt

In [ ]:
total